In [1]:
system("module use /mod/bigelow")
system("module load nlop")

In [2]:
library(viridis)
library(plot3D)
library(stringr)
library(factoextra)
library(FactoMineR)


Loading required package: viridisLite

Warning message:
“no DISPLAY variable so Tk is not available”


ERROR: Error in library(factoextra): there is no package called ‘factoextra’


In [ ]:
install.packages("nloptr")

I have a problem with nloptr on charlie. It works on my computer but not on charlie.

In [ ]:
packages = c("viridis", "plot3D", "stringr",
             "factoextra", "FactoMineR")

package.check <- lapply(
  packages,
  FUN = function(x) {
    if (!require(x, character.only = TRUE)) {
      install.packages(x, dependencies = TRUE)
      library(x, character.only = TRUE)
    }
  }
)



In [ ]:
norm_or_scale <- 3 #1=none, 2=z norming and scaling, 3=mean norming

setwd("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/")

file_name <- "Transcripts_percell_O2_percell_10SAG_genera_stats_biogeochemical_master_no2017.csv"
main_title <- "KO_Transcripts_percell_O2_percell_10SAG_GENstats_biogeochemical_master_no2017"
stats_name <- "KO_Transcripts"
#setwd("~/desktop/")
print(paste("Reading",file_name))
input_data <- read.csv(file=file_name, header=TRUE)

print("Cleaning data")
#keep data upto to but not including the Weighted_avg_O2 column
O2_keep <- str_detect(string=colnames(input_data), pattern="Weighted_avg_O2_consumed_per_cell", negate = FALSE)
O2_col <- which(O2_keep)

#work_data <- input_data[,1:(O2_col-1)]
#get rid of the X, gen, and date columns
#work_data <- work_data[,-(1:3)]

work_data <- input_data[,4:(O2_col-1)]
#replace NA data with zeros
work_data[is.na(work_data)] <- 0

if (norm_or_scale == 1){
  norm_scale <- "raw data"
}

if (norm_or_scale == 2){
  print("z normalizing data")
  work_data_a <- work_data
  #with what's left, scale and center by ROWS?
  work_data <- t(scale(t(work_data_a), center=TRUE, scale=TRUE))
  norm_scale <- "z norming"
}

if (norm_or_scale == 3){
  print("mean normalizing data")
  #normalize each row by its mean
  work_data_a <- work_data
  mean_by_row <- rowMeans(work_data_a)
  for (ijk in 1:nrow(work_data_a)){
    work_data[ijk,] <- work_data_a[ijk,]/mean_by_row[ijk]
  }
  norm_scale <- "mean norming"
}
print("Conducting PCA")
gen_pca <- PCA(work_data, scale.unit=TRUE, graph=FALSE)

phyla_keep <- str_detect(string=colnames(input_data), pattern="Phyla", negate = FALSE)
phyla_col <- which(phyla_keep)
phyla_unique <- unique(input_data[,phyla_col])
n_phyla <- length(phyla_unique)

#class_keep <- str_detect(string=colnames(input_data), pattern="Class", negate = FALSE)
class_col <- phyla_col+1   #which(class_keep)
class_unique <- unique(input_data[,class_col])
n_class <- length(class_unique)

#order_keep <- str_detect(string=colnames(input_data), pattern="Order", negate = FALSE)
order_col <- phyla_col+2 #which(order_keep)
order_unique <- unique(input_data[,order_col])
n_order <- length(order_unique)

#family_keep <- str_detect(string=colnames(input_data), pattern="Family", negate = FALSE)
family_col <- phyla_col+3  #which(family_keep)
family_unique <- unique(input_data[,family_col])
n_family <- length(family_unique)

genus_col <- phyla_col+4  #which(family_keep)
genus_unique <- unique(input_data[,genus_col])
n_genus <- length(genus_unique)

bestclass_col <- phyla_col+6  #which(family_keep)
bestclass_unique <- unique(input_data[,bestclass_col])
n_bestclass <- length(bestclass_unique)
#save original graphic parameter settings
op <- par(no.readonly = TRUE)

for (j in 1:6){
  sym_var <- rep(NA,times=nrow(input_data))
  classification <- paste("Phyla", norm_scale)
  data_unique <- phyla_unique
  data_col <- phyla_col
  sym_unique <- seq_along(class_unique)
  if (j == 1) {
    sym_name <- input_data[,phyla_col]
    for (k in seq_along(phyla_unique)){
      change <- (sym_name == phyla_unique[k])
      sym_var[change] <- k
    }
  }
  else if (j == 2) {
    sym_name <- input_data[,order_col]
    classification <- paste("Order", norm_scale)
    data_unique <- order_unique
    data_col <- order_col
    sym_unique <- -(64+seq_along(order_unique))
    for (k in seq_along(order_unique)){
      change <- (sym_name == order_unique[k])
      sym_var[change] <- -64-k  #use UNICODE plotting symbols (mostly capital letters)
    }
  }
  else if (j == 3) {
    sym_name <- input_data[,class_col]
    classification <- paste("Class", norm_scale)
    data_unique <- class_unique
    data_col <- class_col
    sym_unique <- seq_along(class_unique)
    for (k in seq_along(class_unique)){
      change <- (sym_name == class_unique[k])
      sym_var[change] <- k
    }
  }
  else if (j == 4) {
    classification <- paste("Family", norm_scale)
    sym_name <- input_data[,family_col]
    data_unique <- family_unique
    data_col <- family_col
    sym_unique <- -(64+seq_along(family_unique))
    for (k in seq_along(family_unique)){
      change <- (sym_name == family_unique[k])
      sym_var[change] <- -64-k  #use UNICODE plotting symbols (mostly capital letters)
    }
  }
  else if (j == 5) {
    classification <- paste("Genus", norm_scale)
    sym_name <- input_data[,genus_col]
    data_unique <- genus_unique
    data_col <- genus_col
    sym_unique <- -(64+seq_along(genus_unique))
    for (k in seq_along(genus_unique)){
      change <- (sym_name == genus_unique[k])
      sym_var[change] <- -64-k  #use UNICODE plotting symbols (mostly capital letters)
    }
  }
  else if (j == 6) {
    classification <- paste("Best_classification", norm_scale)
    sym_name <- input_data[,bestclass_col]
    data_unique <- bestclass_unique
    data_col <- bestclass_col
    sym_unique <- -(64+seq_along(bestclass_unique))
    for (k in seq_along(bestclass_unique)){
      change <- (sym_name == genus_unique[k])
      sym_var[change] <- -64-k  #use UNICODE plotting symbols (mostly capital letters)
    }
  } 
  print(paste("Working on graphs by ", classification, sep=""))
  
  #O2 consumption box-and-whisker plot
  file_name <- paste("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/PCA/",classification," O2 consumption.pdf",sep="")
  pdf(file=file_name, width=5.30, height=4.500)
  par <- op
  par(mai=c(2.0,0.82,0.5,0.4))
  abc <- boxplot(log10(input_data[,O2_col])~input_data[,data_col],
                 xlab="", ylab="log10(O2 consumption)", xaxt = "n", 
                 main=paste("O2 consumption by", classification)) 
  axis(side=1, las=2, at=seq_along(abc$names), labels=abc$names, xpd=NA, cex.axis=0.7)
  dev.off()
  
  #Eigenvalue plots
  file_name <- paste("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/PCA/",classification," ",stats_name," eig.pdf",sep="")
  pdf(file=file_name, width=5.30, height=4.500)
  par <- op
  abc <- barplot(height=gen_pca$eig[1:10,2], 
                 names.arg=paste("PC ",1:10, sep=""), cex.names=0.7,
                 xlab="PC", ylab="% Explained By PC Variance", 
                 main=paste(stats_name,"PC Variance"))
  ax_limits <- par("usr")
  delta_y <- ax_limits[4]-ax_limits[3]
  text(x=abc, y=gen_pca$eig[1:10,2]+0.01*delta_y, adj=c(0.5,0), 
       labels=round(gen_pca$eig[1:10,2], digits=2), xpd=NA, cex=0.8)
  dev.off()
  
  #3D (PC1 vs PC2 vs O2) consumption plot
  file_name <- paste("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/PCA/",classification," ",stats_name," 3D (PC1, PC2, O2).pdf",sep="")
  pdf(file=file_name, width=5.30, height=4.500)
  par <- op
  par(mai=c(1.02,0.82,0.82,1.5))
  points3D(x=gen_pca$ind$coord[,1], y=gen_pca$ind$coord[,2], z=log10(input_data[,O2_col]),
           xlab="PC 1", ylab="PC 2", zlab="log10(O2 consumed per cell)", 
           cex.axis=0.6, cex.main=0.7, cex.lab=0.6,
           main=paste(classification,stats_name,"PC1 vs. PC2 vs. O2"),
           pch=sym_var, 
           col=viridis(401), colvar=log10(input_data[,O2_col]), 
           colkey=list(side=4, length=1, width=1, cex.axis=0.8, dist=-0.02))
  ax_limits <- par("usr")
  legend(x=ax_limits[2]+0.275*(ax_limits[2]-ax_limits[1]), 
         y=ax_limits[4]+0.01*(ax_limits[4]-ax_limits[3]), 
         legend=data_unique, 
         pch=sym_unique, col="black", cex=0.5, xpd=NA)
  dev.off()
  
  #PC1 vs PC2 plot
  file_name <- paste("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/PCA/",classification," ",stats_name," PC1 vs PC2.pdf",sep="")
  pdf(file=file_name, width=5.30, height=4.500)
  par <- op
  par(mai=c(1.02,0.82,0.82,2.3))
  points2D(x=gen_pca$ind$coord[,1], y=gen_pca$ind$coord[,2], type="p",
           xlab="PC 1", ylab="PC 2", 
           main=paste(classification,stats_name,"PC1 vs. PC2"),
           pch=sym_var, 
           col=viridis(401), colvar=log10(input_data[,O2_col]), colkey=TRUE)
  ax_limits <- par("usr")
  legend(x=ax_limits[2]+0.5*(ax_limits[2]-ax_limits[1]), 
         y=ax_limits[4]+0.1*(ax_limits[4]-ax_limits[3]), 
         legend=data_unique, 
         pch=sym_unique, col="black", cex=0.7, xpd=NA)
  dev.off()
  
  for (i in 1:5){
    file_name <- paste("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/PCA/",classification," ",stats_name," PC",i," vs O2.pdf",sep="")
    pdf(file=file_name, width=5.30, height=4.500)
    par <- op
    par(mai=c(1.02,0.82,0.82,2.3))
    points2D(x=gen_pca$ind$coord[,i], y=log10(input_data[,O2_col]),
             xlab=paste("PC",i,sep=""), ylab="log10(O2 consumed per cell)", 
             main=paste(classification," ",stats_name," PC",i," vs O2", sep=""),
             pch=sym_var, 
             col=viridis(401), colvar=log10(input_data[,O2_col]), 
             colkey=TRUE)
    ax_limits <- par("usr")
    legend(x=ax_limits[2]+0.5*(ax_limits[2]-ax_limits[1]), 
           y=ax_limits[4]+0.1*(ax_limits[4]-ax_limits[3]), 
           legend=data_unique, 
           pch=sym_unique, col="black", cex=0.7, xpd=NA)
    dev.off()
    
    #PC vs contributions plot
    pdf(file=paste("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/PCA/",classification," ",stats_name," PC",i,".pdf",sep=""), 
        width=5.30, height=4.500)
    contribs <- gen_pca$var$contrib
    sort1 <- order(-contribs[,i])
    ht <- contribs[sort1[1:10],i]
    gen_label <- row.names(contribs[sort1[1:10],])
    par <- op
    par(oma=c(3.5,0,0,0))
    abc <- barplot(height=ht, ylab="% Contributed to PC",i, names.arg="", 
                   main=paste(classification," ",stats_name," PC",i,sep=""))
    ax_limits <- par("usr")
    delta_y <- ax_limits[4]-ax_limits[3]
    text(x=0.5,y=seq(from=ax_limits[3]-0.025*delta_y, 
                     to=ax_limits[3]-0.35*delta_y, 
                     length.out=10), 
         labels=gen_label, xpd=NA, cex=0.5, adj=c(0,0.5))
    text(x=abc, y=ht+0.01*delta_y, adj=c(0.5,0), 
         labels=round(ht, digits=3), xpd=NA, cex=0.8)
    dev.off()
  }
}
par <- op

